### Install Runhouse

In [ ]:
!pip install git+https://github.com/run-house/runhouse.git@latest_patch

In [ ]:
import runhouse as rh

INFO | 2022-12-13 05:38:24,002 | Loaded Runhouse config from /root/.rh/config.yaml


### Login to Runhouse to load in secrets.

In [ ]:
# You can add token=<your token> if you want to be able to run this without pasting into stdin
rh.login(download_secrets=True, download_config=True, interactive=True)

In [ ]:
# Only if you're using GCP and running inside Colab!
!gcloud init
!gcloud auth application-default login
!cp -r /content/.config/* ~/.config/gcloud

In [ ]:
# Check that secrets are loaded in properly and at least one cloud is ready to use.
!sky check

# Dreambooth Training

Start by creating our training service.

In [ ]:
gpu = rh.cluster(name='rh-a10x', instance_type='A100:1')  # On GCP and Azure
# gpu = rh.cluster(name='rh-a10x', instance_type='g5.2xlarge', provider='aws')  # On AWS

training_function_gpu = rh.send(
    fn='https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/train_dreambooth.py:main',
    hardware=gpu,
    reqs=['pip:./diffusers'],
    name='train_dreambooth')
training_function_gpu.run_setup(['mkdir dreambooth'])
gpu.run_python(['import torch; torch.backends.cuda.matmul.allow_tf32 = True; '
                'torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True'])

Next, we need to upload some images.

In [ ]:
from google.colab import files
import shutil
from pathlib import Path

uploaded = files.upload()

Now we'll send those images to our cluster.

In [ ]:
input_images_dir = 'images'
images_path = Path(input_images_dir)
images_path.mkdir(exist_ok=True)

for filename in uploaded.keys():
  shutil.move(filename, images_path / filename)

remote_image_dir = 'dreambooth/instance_images'
rh.folder(url=input_images_dir).to(fs=gpu, url=remote_image_dir)

Now we'll generate the arguments into the training function (and call this function on the cluster to avoid having to clone it down locally).

In [ ]:
create_train_args = rh.send(
    fn='https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/train_dreambooth.py:parse_args',
    hardware=gpu, reqs=[])
train_args = create_train_args(input_args=['--pretrained_model_name_or_path', 'stabilityai/stable-diffusion-2-base',
                                            '--instance_data_dir', remote_image_dir,
                                            '--instance_prompt', f'a photo of sks {class_name}'])
train_args.train_text_encoder = True
train_args.class_data_dir = 'dreambooth/class_images'
train_args.output_dir = 'dreambooth/output'
train_args.mixed_precision = 'bf16'
train_args.with_prior_preservation = True
train_args.prior_loss_weight = 1.0
train_args.class_prompt = f"a photo of {class_name}"
train_args.resolution = 512
train_args.train_batch_size = 4
train_args.gradient_checkpointing = True
train_args.learning_rate = 1e-6
train_args.lr_scheduler = "constant"
train_args.lr_warmup_steps = 0
train_args.num_class_images = 200
train_args.checkpointing_steps = 400
# train_args.resume_from_checkpoint = 'latest'
train_args.max_train_steps = 1200

And initiate training:

In [ ]:
training_function_gpu(train_args)

# Inference

Now we can use our existing Stable Diffusion service to run inferences on this model:

In [ ]:
generate_gpu = rh.Send.from_name(name='sd_generate')

In [ ]:
model_path = 'dreambooth/output'
images = generate_gpu(my_prompt,
                      model_id=model_path,
                      num_images=4, guidance_scale=7.5,
                      steps=100)

In [ ]:
[image.show() for image in images]